In [22]:
import pykep as pk
import numpy as np
pk.util.load_spice_kernel("/home/m/mmk40/Desktop/Planet Position/de431.bsp")
from pykep.orbit_plots import plot_planet, plot_lambert
from pykep import AU, DAY2SEC, SEC2DAY, DAY2YEAR
import pygmo as pg
from datetime import datetime, timedelta
import math

In [23]:
# Load planets
earth = pk.planet.spice('EARTH' , 'SUN', 'ECLIPJ2000', 'NONE')
jupiter = pk.planet.spice('JUPITER BARYCENTER', 'SUN', 'ECLIPJ2000', 'NONE' )

# Set launch window
start_date = pk.epoch_from_string("2035-01-01 00:00:00")
end_date = pk.epoch_from_string("2050-01-01 00:00:00")

# Parameters
launch_step = 30     # days between launch trials
tof_range = (300, 1500)  # in days, time of flight range

In [42]:
# Results list
results = []

# Scan through departure dates
t_dep = start_date
while t_dep.mjd2000 < end_date.mjd2000:
    rE, vE = earth.eph(t_dep)

    for tof_days in range(tof_range[0], tof_range[1], 30):  # try multiple TOFs
        try:
            t_arr = pk.epoch(t_dep.mjd2000 + tof_days)
            rJ, vJ = jupiter.eph(t_arr)

            dt_sec = tof_days * pk.DAY2SEC
            l1 = pk.lambert_problem(r1=rE, r2=rJ, tof=dt_sec, mu=pk.MU_SUN, max_revs=2)

            v1 = np.array(l1.get_v1()[0])
            v2 = np.array(l1.get_v2()[0])
            v_inf = (v1 - np.array(vE))/1000
            delta_v = (np.linalg.norm(v1 - np.array(vE)))/1000 
            v_arrival   = l1.get_v2()[0] # velocity of spacecraft at jupiter arrival
            hyperbolic_v_arrival   = (np.linalg.norm(np.array(v_arrival) - np.array(vJ)))/1000   # km/s  # substracting velocities and taking magnitude
            C3 = np.linalg.norm(v_inf) ** 2  # in km²/s²
            

            # Calculating Flyby speed

            rp = 9 * 69911e3 # m  # radius of periapsis for the flyby trajectory (change this)

            mu = 6.67e-11 * 1.898e27 # m3/s2  # gravitationalparameter for jupiter

            v_in = hyperbolic_v_arrival * 1000 # m/s # Jupiter arrival speed

            # Turning Angle
            phi = np.degrees(np.pi - 2 * np.acos (1/(((v_in**2)*rp)/mu + 1)))
            # print (np.pi) # Degree/radian check
            # print (f" Turning angle: {phi:.2f} degrees") # degrees


            # Law of Cosines to find the v_out
            #print(vJ) # vector/scalar check
            vJ_mag = np.linalg.norm(vJ)
            #print(vJ_mag)

            exit_speed = (math.sqrt((vJ_mag**2) + (v_in**2) - 2*vJ_mag*v_in*np.cos(phi)))/1000 # km/s
            #print(f"Flyby speed: {(exit_speed - hyperbolic_v_arrival):.2f} km/s")
            #print (f"Jupiter Departure Speed: {exit_speed:.2f} km/s")

            # Time to Heliopause from Jupiter flyby

            d = 194.8*AU # 200 AU in meters
            s = exit_speed*1000 # speed of space craft at jupiter exit in m/s 

            # print (d) .pk check

            time_to_200AU = (d/s)*SEC2DAY*DAY2YEAR
            total_time = time_to_200AU + tof_days/365
            flyby_speed = exit_speed - hyperbolic_v_arrival

            #print(f"Time to reach Jupiter: {dt*SEC2DAY*DAY2YEAR: .2f} years")
            #print(f"Time to 200 AU from Jupiter Flyby: {time_to_200AU:.2f} years")
            #print(f"Total Time to reach 200 AU from launch: {time_to_200AU + dt*SEC2DAY*DAY2YEAR : .2f} years")

            d1 = 84.8*AU
            s1 = exit_speed*1000

            time_to_90AU = (d1/s1)*SEC2DAY*DAY2YEAR # time to reach 90 AU

            d2 = 114.8*AU
            s2 = exit_speed*1000

            time_to_120AU = (d2/s2)*SEC2DAY*DAY2YEAR # time to reach 120 AU

            # Keplarian elements to Outgoing Asymptote
            # Function to convert v_inf_vec to RA and Dec
            def vector_to_ra_dec(vec):
                x, y, z = vec
                r = np.linalg.norm(vec)
                ra = np.arctan2(y, x)
                dec = np.arcsin(z / r)
                return np.degrees(ra) % 360, np.degrees(dec)



            # Compute outgoing asymptote direction
            v_inf_vec = (np.array(v_arrival) - np.array(vJ))  # Outgoing Asymptote Conditions at Jupiter arrival 
            v_inf_mag = np.linalg.norm(v_inf_vec) /1000 # km/s
            Outgoing_C3 = v_inf_mag**2 #km²/s²
            ra_inf, dec_inf = vector_to_ra_dec(v_inf_vec)

            

            if C3 <= 100:
                if flyby_speed > 0:
                    if total_time <= 75:

                        # Get Keplerian elements of transfer orbit
                        kep = pk.ic2par(rE, v1, pk.MU_SUN)  # r, v at arrival → orbital elements

                        results.append({
                            'departure': str(t_dep),
                            'arrival': str(t_arr),
                            'tof_days': tof_days,
                            'C3': C3,
                            'delta_v': delta_v,
                            'sma': kep[0],   # semi-major axis
                            'ecc': kep[1],   # eccentricity
                            'inc': np.degrees(kep[2]),  # inclination (rad → deg)
                            'raan': np.degrees(kep[3]),
                            'argp': np.degrees(kep[4]),
                            'ta': np.degrees(kep[5]),
                            'turning angle': phi,
                            'flyby speed': flyby_speed,
                            'jupiter departure speed': exit_speed,
                            'time_to_90AU': time_to_90AU,
                            'time_to_120AU': time_to_120AU,
                            'jupiter flyby to 200 au': time_to_200AU,
                            'total_time' : total_time,
                            'Outgoing_C3' : Outgoing_C3,
                            'ra_inf': ra_inf,
                            'dec_inf': dec_inf,
                    
                        })

        except RuntimeError as e:
            # Lambert solver may fail for some configurations; skip them
            continue

    t_dep = pk.epoch(t_dep.mjd2000 + launch_step)

In [43]:
# Sort by C3 (ascending)
results.sort(key=lambda x: x['total_time'])

# Print trajectories
for res in results[:]:
    print(f"Launch: {res['departure']}, Arrival: {res['arrival']}, C3: {res['C3']:.2f} km²/s², Delta v: {res['delta_v']:.2f} km/s \n\nTime to reach Jupiter: {res['tof_days']/365:.2f} years, Time to 200 AU from Jupiter Flyby: {res['jupiter flyby to 200 au']: .2f} years, Total Time to 200 AU from Launch: {res['total_time']:.2f} years\n")
    print(f"Time to reach 90 AU: {res['time_to_90AU']: .2f} years, Time to reach 120 AU: {res['time_to_120AU']: .2f} years\n")
    print(f"Turning angle: {res['turning angle']: .2f}° | Flyby Speed: {res['flyby speed']: .2f} km/s \n ")
    print(f"Elements for EARTH -> JUPITER → SMA: {res['sma']/1000:.3f} km, ECC: {res['ecc']:.4f}, INC: {res['inc']:.2f}°\n")
    print(f"Elements EARTH -> JUPITER → RAAN: {res['raan']:.3f}° , AP: {res['argp']:.4f}°, TA: {res['ta']:.2f}°\n\n\n")
    #print(f"GMAT Outgoing Asymptote:")
    #print(f"  Outgoing_C3: {res['Outgoing_C3']:.2f} km²/s² | RA: {res['ra_inf']:.2f} deg | DEC: {res['dec_inf']:.2f} deg\n\n\n ")
  

Launch: 2043-Feb-18 00:00:00, Arrival: 2044-Sep-10 00:00:00, C3: 98.75 km²/s², Delta v: 9.94 km/s 

Time to reach Jupiter: 1.56 years, Time to 200 AU from Jupiter Flyby:  38.47 years, Total Time to 200 AU from Launch: 40.03 years

Time to reach 90 AU:  16.75 years, Time to reach 120 AU:  22.67 years

Turning angle:  78.32° | Flyby Speed:  13.17 km/s 
 
Elements for EARTH -> JUPITER → SMA: 622721025.402 km, ECC: 0.7634, INC: 1.41°

Elements EARTH -> JUPITER → RAAN: 328.506° , AP: 187.1696°, TA: -2.59°



Launch: 2044-Apr-13 00:00:00, Arrival: 2045-Nov-04 00:00:00, C3: 98.58 km²/s², Delta v: 9.93 km/s 

Time to reach Jupiter: 1.56 years, Time to 200 AU from Jupiter Flyby:  39.50 years, Total Time to 200 AU from Launch: 41.07 years

Time to reach 90 AU:  17.20 years, Time to reach 120 AU:  23.28 years

Turning angle:  84.62° | Flyby Speed:  13.49 km/s 
 
Elements for EARTH -> JUPITER → SMA: 564003636.161 km, ECC: 0.7370, INC: 1.77°

Elements EARTH -> JUPITER → RAAN: 23.194° , AP: 166.6990

In [47]:
def ecliptic_to_cartesian(lambda_deg, beta_deg, distance_km):
    lam_rad = np.deg2rad(lambda_deg)
    beta_rad = np.deg2rad(beta_deg)

    x = distance_km * np.cos(beta_rad) * np.cos(lam_rad) #cos(lat)cos(long)
    y = distance_km * np.cos(beta_rad) * np.sin(lam_rad)
    z = distance_km * np.sin(beta_rad)
    return np.array([x, y, z])


lam_deg = 240 # Latitude of the Nose from Freeman's
beta_deg = 7.5 # Longitude of the Nose from Freeman's
heliopause_distance = 120*pk.AU 

rH = ecliptic_to_cartesian(lam_deg, beta_deg, heliopause_distance) # position of the nose of the helioshpere
#print("Cartesian position (heliocentric, J2000):", rH)

# Set launch window
start_date = pk.epoch_from_string("2039-01-01 00:00:00")
end_date = pk.epoch_from_string("2046-01-01 00:00:00")

# Parameters
launch_step = 30     # days between launch trials
tof_range = (12000, 24000)  # in days, time of flight range

In [48]:
# Results list
results = []

# Scan through departure dates
t_dep = start_date
while t_dep.mjd2000 < end_date.mjd2000:
    rJ, vJ = jupiter.eph(t_dep)
    
    for tof_days in range(tof_range[0], tof_range[1], 30):  # try multiple TOFs
        try:
            t_arr = pk.epoch(t_dep.mjd2000 + tof_days)
            dt_sec = tof_days * pk.DAY2SEC
            l2 = pk.lambert_problem(r1=rJ, r2=rH, tof=dt_sec, mu=pk.MU_SUN, max_revs=2)

            v1 = np.array(l2.get_v1()[0])
            v2 = np.array(l2.get_v2()[0])
            v_inf = (v1 - np.array(vJ))/1000
            C3 = np.linalg.norm(v_inf) ** 2  # in km²/s²
            if C3 <= 250:
                # Get Keplerian elements of transfer orbit
                kep = pk.ic2par(rJ, v1, pk.MU_SUN)  # r, v at arrival → orbital elements

                results.append({
                    'departure': str(t_dep),
                    'arrival': str(t_arr),
                    'tof_days': tof_days,
                    'C3': C3,
                    'sma': kep[0],   # semi-major axis
                    'ecc': kep[1],   # eccentricity
                    'inc': np.degrees(kep[2]),  # inclination (rad → deg)
                    'raan': np.degrees(kep[3]),
                    'argp': np.degrees(kep[4]),
                    'ta': np.degrees(kep[5]),
                })

        except RuntimeError as e:
            # Lambert solver may fail for some configurations; skip them
            continue

    t_dep = pk.epoch(t_dep.mjd2000 + launch_step)

In [49]:
#Sort by C3 (ascending)
results.sort(key=lambda x: x['tof_days'])

# Print trajectories
for res in results[:]:
    print(f"Launch: {res['departure']}, Arrival: {res['arrival']}, C3: {res['C3']:.2f} km²/s², Time to reach Nose: {res['tof_days']/365: .2f} years\n")
    print(f"Elements for JUPITER -> HELIOPAUSE → SMA: {res['sma']/1000:.3f} km, ECC: {res['ecc']:.4f}, INC: {res['inc']:.2f}°\n")
    print(f"Elements JUPITER -> HELIOPAUSE → RAAN: {res['raan']:.3f}° , AP: {res['argp']:.4f}°, TA: {res['ta']:.2f}°\n\n\n")

Launch: 2039-Jan-01 00:00:00, Arrival: 2071-Nov-09 00:00:00, C3: 141.14 km²/s², Time to reach Nose:  32.88 years

Elements for JUPITER -> HELIOPAUSE → SMA: 519006723.889 km, ECC: 2.5166, INC: 7.73°

Elements JUPITER -> HELIOPAUSE → RAAN: 135.957° , AP: 354.2819°, TA: 8.03°



Launch: 2039-Jan-31 00:00:00, Arrival: 2071-Dec-09 00:00:00, C3: 143.42 km²/s², Time to reach Nose:  32.88 years

Elements for JUPITER -> HELIOPAUSE → SMA: 520711299.829 km, ECC: 2.5030, INC: 7.67°

Elements JUPITER -> HELIOPAUSE → RAAN: 137.964° , AP: 352.1452°, TA: 9.65°



Launch: 2039-Mar-02 00:00:00, Arrival: 2072-Jan-08 00:00:00, C3: 146.34 km²/s², Time to reach Nose:  32.88 years

Elements for JUPITER -> HELIOPAUSE → SMA: 522445119.075 km, ECC: 2.4872, INC: 7.61°

Elements JUPITER -> HELIOPAUSE → RAAN: 139.981° , AP: 349.9722°, TA: 11.27°



Launch: 2039-Apr-01 00:00:00, Arrival: 2072-Feb-07 00:00:00, C3: 149.90 km²/s², Time to reach Nose:  32.88 years

Elements for JUPITER -> HELIOPAUSE → SMA: 524205204.98